<a href="https://colab.research.google.com/github/doronschwartz/NLP/blob/main/HW2/HW2_Part3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using pre-trained word embeddings

**Author:** [fchollet](https://twitter.com/fchollet)<br>
**Date created:** 2020/05/05<br>
**Last modified:** 2020/05/05<br>
**Description:** Text classification on the Newsgroup20 dataset using pre-trained GloVe word embeddings.

Taken from:
https://keras.io/examples/nlp/pretrained_word_embeddings/

Modified by Avi Rosenfeld on 2023/21/02

## Setup

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

## Introduction

In this example, we show how to train a text classification model that uses pre-trained
word embeddings.

We'll work with the Newsgroup20 dataset, a set of 20,000 message board messages
belonging to 20 different topic categories.

For the pre-trained word embeddings, we'll use
[GloVe embeddings](http://nlp.stanford.edu/projects/glove/).

However, we will use Gensim's pretrained models.  More on all models can be found at:

https://radimrehurek.com/gensim/models/word2vec.html#pretrained-models

## Load pre-trained word embeddings

Let's download a pre-trained GloVe embeddings from Gensim.  This one is a model with 200 nodes. Feel free to try the other options with 50-dimensional,
100-dimensional, 200-dimensional, 300-dimensional vectors. Also, you can put in the Word2Vec ones just as easily!

In [ ]:
import gensim.downloader as api
glove_model = api.load("glove-wiki-gigaword-200")
embedding_dim = glove_model.vector_size

In [ ]:
print(embedding_dim)

200


## Download the Newsgroup20 data

Note that as opposed to our previous notebook, here we download the entire 20_newsgroup data.

In [ ]:
from sklearn.datasets import fetch_20newsgroups
#newsgroups_all = fetch_20newsgroups(subset='all', shuffle=True, random_state=42, remove=('headers', 'footers', 'quotes'))
cats = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
#newsgroups_all = fetch_20newsgroups(subset='all',categories=cats)
newsgroups_all = fetch_20newsgroups(subset='all',categories=cats, remove=('headers', 'footers', 'quotes'))

## Let's take a look at the data

As you can see, there are header lines that are leaking the file's category, either
explicitly (the first line is literally the category name), or implicitly, e.g. via the
`Organization` filed. We get rid of the headers with the built-in filter.  

In [ ]:
samples = newsgroups_all.data
labels =  newsgroups_all.target
class_names = newsgroups_all.target_names

print("Classes:", class_names)
print("Number of samples:", len(samples))

Classes: ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']
Number of samples: 3387


## Shuffle and split the data into training & validation sets

In [ ]:
# Shuffle the data
seed = 1337
rng = np.random.RandomState(seed)
rng.shuffle(samples)
rng = np.random.RandomState(seed)
rng.shuffle(labels)

# Extract a training & validation split
validation_split = 0.2
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
val_samples = samples[-num_validation_samples:]
train_labels = labels[:-num_validation_samples]
val_labels = labels[-num_validation_samples:]

In [ ]:
#train_samples = train_samples.astype(str)

## Create a vocabulary index

Let's use the `TextVectorization` to index the vocabulary found in the dataset.
Later, we'll use the same layer instance to vectorize the samples.

Our layer will only consider the top 20,000 words, and will truncate or pad sequences to
be actually 200 tokens long.

In [ ]:
from tensorflow.keras.layers import TextVectorization

vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectorizer.adapt(text_ds)

You can retrieve the computed vocabulary used via `vectorizer.get_vocabulary()`. Let's
print the top 5 words:

In [ ]:
vectorizer.get_vocabulary()[:5]

['', '[UNK]', 'the', 'to', 'of']

Let's vectorize a test sentence:

In [ ]:
output = vectorizer([["the cat sat on the man"]])
output.numpy()[0, :6]

array([    2, 11969,  4109,    17,     2,   339])

As you can see, "the" gets represented as "2". Why not 0, given that "the" was the first
word in the vocabulary? That's because index 0 is reserved for padding and index 1 is
reserved for "out of vocabulary" tokens.

Here's a dict mapping words to their indices:

In [ ]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

Now, let's prepare a corresponding embedding matrix that we can use in a Keras
`Embedding` layer. It's a simple NumPy matrix where entry at index `i` is the pre-trained
vector for the word of index `i` in our `vectorizer`'s vocabulary.

Hint: I found you can add stopwords or other filters here. I personally used the one from:

from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

As you can see, we obtain the same encoding as above for our test sentence:

In [ ]:
test = ["the", "man", "sat", "on", "the", "cat"]


Now let's encode the 20 Newsgroup dataset using our preloaded word embeddings!

In [ ]:
vocab_size = len(word_index)
num_tokens = len(voc) + 2
embedding_matrix = np.zeros((num_tokens, embedding_dim))

hits = 0
misses = 0

# Prepare embedding matrix
for word, i in word_index.items():
    if word in glove_model:
      hits += 1
      embedding_matrix[i] = glove_model[word]
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))


Converted 15832 words (4168 misses)


In [ ]:
print(word_index.items())
print(print(embedding_matrix[10])) #Note that UNK is like OOV from the Lecture

dict_items([('', 0), ('[UNK]', 1), ('the', 2), ('to', 3), ('of', 4), ('and', 5), ('a', 6), ('is', 7), ('in', 8), ('that', 9), ('i', 10), ('it', 11), ('for', 12), ('you', 13), ('this', 14), ('be', 15), ('are', 16), ('on', 17), ('not', 18), ('as', 19), ('have', 20), ('or', 21), ('with', 22), ('if', 23), ('from', 24), ('but', 25), ('by', 26), ('was', 27), ('at', 28), ('can', 29), ('an', 30), ('they', 31), ('will', 32), ('there', 33), ('what', 34), ('all', 35), ('would', 36), ('do', 37), ('one', 38), ('about', 39), ('we', 40), ('so', 41), ('your', 42), ('which', 43), ('some', 44), ('has', 45), ('its', 46), ('my', 47), ('he', 48), ('any', 49), ('no', 50), ('more', 51), ('other', 52), ('me', 53), ('image', 54), ('also', 55), ('who', 56), ('dont', 57), ('people', 58), ('out', 59), ('their', 60), ('space', 61), ('like', 62), ('god', 63), ('than', 64), ('just', 65), ('only', 66), ('were', 67), ('know', 68), ('then', 69), ('his', 70), ('them', 71), ('does', 72), ('how', 73), ('when', 74), ('up',

Next, we load the pre-trained word embeddings matrix into an `Embedding` layer.

Note that we set `trainable=False` so as to keep the embeddings fixed (we don't want to update them during training).  

This acts as the "transfer learning" aspect of this model.


In [ ]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

## Build the model

A simple 1D convnet with global max pooling and a classifier at the end.  The different convolutional and pooling layers are meant to act as a type of feature selection, only focusing on the important embeddings (words).  However, I have found that some feature selection done in advance still helps by either speeding up performance and / or model speed.

In [ ]:
from tensorflow.keras import layers

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(len(class_names), activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 200)         4000400   
                                                                 
 conv1d (Conv1D)             (None, None, 128)         128128    
                                                                 
 max_pooling1d (MaxPooling1  (None, None, 128)         0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, None, 128)         82048     
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, None, 128)         0         
 g1D)                                                        

## Train the model

First, convert our list-of-strings data to NumPy arrays of integer indices. The arrays
are right-padded.

In [ ]:
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(val_labels)

We use categorical crossentropy as our loss since we're doing softmax classification.
Moreover, we use `sparse_categorical_crossentropy` since our labels are integers.

In [ ]:
model.compile(
    loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["acc"],
)
model.fit(x_train, y_train, batch_size=128, epochs=30, validation_data=(x_val, y_val))

Epoch 1/30
22/22 [==============================] - 7s 57ms/step - loss: 1.2961 - acc: 0.3808 - val_loss: 1.0758 - val_acc: 0.5155
Epoch 2/30
22/22 [==============================] - 0s 12ms/step - loss: 0.9139 - acc: 0.5878 - val_loss: 0.7693 - val_acc: 0.6455
Epoch 3/30
22/22 [==============================] - 0s 11ms/step - loss: 0.6893 - acc: 0.6793 - val_loss: 0.6088 - val_acc: 0.6987
Epoch 4/30
22/22 [==============================] - 0s 10ms/step - loss: 0.5846 - acc: 0.7255 - val_loss: 0.6185 - val_acc: 0.6942
Epoch 5/30
22/22 [==============================] - 0s 12ms/step - loss: 0.5079 - acc: 0.7506 - val_loss: 0.7054 - val_acc: 0.6544
Epoch 6/30
22/22 [==============================] - 0s 12ms/step - loss: 0.4566 - acc: 0.7856 - val_loss: 0.5666 - val_acc: 0.6883
Epoch 7/30
22/22 [==============================] - 0s 11ms/step - loss: 0.3801 - acc: 0.8203 - val_loss: 0.6048 - val_acc: 0.6809
Epoch 8/30
22/22 [==============================] - 0s 11ms/step - loss: 0.3309 - a

## Export an end-to-end model

Now, we may want to export a `Model` object that takes as input a string of arbitrary
length, rather than a sequence of indices. It would make the model much more portable,
since you wouldn't have to worry about the input preprocessing pipeline.

Our `vectorizer` is actually a Keras layer, so it's simple:

In [ ]:
string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = model(x)
end_to_end_model = keras.Model(string_input, preds)

probabilities = end_to_end_model.predict(
    [["this message is about computer graphics and 3D modeling"]]
)
print("The categories are " + str(class_names))
print("The probabilities for the model are " + str(probabilities))

class_names[np.argmax(probabilities[0])]

1/1 [==============================] - 0s 329ms/step
The categories are ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']
The probabilities for the model are [[6.0392282e-20 1.0000000e+00 4.0591908e-19 2.3149380e-21]]


'comp.graphics'